In [12]:
from google.colab import drive
drive.mount('/content/drive')
import glob
import os
os.environ["WANDB_SILENT"] = "true"
import zipfile
import pandas as pd
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
# add to path
import sys
sys.path.append('/content/drive/Othercomputers/pc/train/')
import trainer
import misc
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [13]:
!pip install wandb
import wandb
wandb.login()

True

In [14]:
# Unzip data
zip_path = 'drive/MyDrive/lfi/dataset_us.zip'
extract_path = 'data'

# Ensure the destination folder exists
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [15]:
# Testset path
test_set_path = "data/dataset_us/test"

results_path = "drive/Othercomputers/pc/train/results/test_set_results"

In [16]:
# Get final models
models_and_log_path = "drive/Othercomputers/pc/train/results/colab"
final_models_paths = glob.glob(models_and_log_path + "/*_final.pth")

In [17]:
# # For testing only a subset
# final_models_paths = [
#    'drive/Othercomputers/pc/train/results/colab/bl_enhanced_2_cw_final.pth',
#    'drive/Othercomputers/pc/train/results/colab/bl_cw_final.pth']

In [18]:
if False:
    models_results = []
    for model_path in sorted(final_models_paths):
        run_name = os.path.basename(model_path).replace("_final.pth", "")
        print(f"\n *************** {run_name} ***************")

        log_name = run_name + "_log.json"
        log = misc.load_from_json(os.path.join(models_and_log_path, log_name))
        training_params = log["params"]

        r = trainer.evaluate_model(model_file_path=model_path,
                                  test_set_path=test_set_path,
                                  batch_size=128,
                                  num_workers=10,
                                  make_detailed_predictions=True)
        # r is a dict like
        # {"labels": all_labels -> list,
        #  "predictions": all_preds -> list
        #  "classes": test_set.classes -> ['non_smile', 'smile']
        #  "detailed_predictions": detailed_predictions -> dict( file, true_label, predicted_label, probabilities)

        df_temp = pd.DataFrame(r["detailed_predictions"])
        df_temp.to_csv(f"{results_path}/{run_name}_predictions.csv", index=False)

        accuracy = accuracy_score(r["labels"], r["predictions"])
        precision, recall, f1, _ = precision_recall_fscore_support(
            r["labels"], r["predictions"], average="binary")
        conf_matrix = confusion_matrix(r["labels"], r["predictions"])

        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1: {f1}")

        # Merge results and training parameters for later overview
        act_model_results = {
            "run_name": run_name,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "conf_matrix": list(conf_matrix)} | training_params

        models_results.append(act_model_results)

        # Logging all this to wandb
        wandb.init(project="LfI24_test", name=run_name,
                  settings=wandb.Settings(console='off'))
        wandb.log(act_model_results)
        wandb.finish()

        # Plot relative confusion matrix
        plt.figure(figsize=(3.3, 3))
        sns.heatmap(conf_matrix / conf_matrix.sum(axis=1)[:, None], annot=True,
                    fmt=".2%", cmap="Blues",
                    xticklabels=r['classes'], yticklabels=r['classes'],
                    cbar=False)
        plt.title(f"Model: {run_name}\n")
        plt.ylabel("True")
        plt.xlabel("Predicted")
        plt.tight_layout()
        plt.savefig(f"{results_path}/{run_name}_confusion.jpg")
        plt.show()

        # Sample plot of images with labels, predictions and confidence ***********
        # Get first 5 and last 5 entries of detailed_predictions
        samples = r["detailed_predictions"][:5] + r["detailed_predictions"][-5:]

        # Grid of 2x5 images
        fig, axs = plt.subplots(2, 5, figsize=(17, 9))
        axs = axs.ravel()

        for i, s in enumerate(samples):
            c0, c1 = s["probabilities"][0], s["probabilities"][1]

            if s["true_label"] == "non_smile":
                image_path = os.path.join(test_set_path, "non_smile", s["file"])
            else:
                image_path = os.path.join(test_set_path, "smile", s["file"])

            img = Image.open(image_path)
            axs[i].imshow(img)
            axs[i].set_title(f"Tr: {s['true_label']}, Pr: {s['predicted_label']}\n"
                            f"ns/s: {c0:.2f}/{c1:.2f}")
            axs[i].axis('off')
        fig.suptitle(f"{run_name}")
        fig.tight_layout()
        plt.savefig(f"{results_path}/{run_name}_samples.jpg")
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# # Models_results to dataframe
# df = pd.DataFrame(models_results)

# # Pickle the dataframe
# df.to_pickle(results_path + "/testset_results.pkl")

# # Save the dataframe as csv
# df.to_csv(results_path + "/testset_results.csv", index=False)